In [ ]:
from __future__ import division, unicode_literals, print_function  # for compatibility with Python 2 and 3

import matplotlib.pyplot as plt # matplotlib package for figure plotting
import numpy as np # numpy package for array handling
import pandas as pd # pandas package for dataframe handling
import trackpy as tp # trackpy itself
import tifffile as tiff # tifffile package for .tif file handling
import os # os package for directory and file handling

%matplotlib inline 
# this line allows us to plot inside the notebook

In [ ]:
img_path = os.getcwd() + 'filename.tif' # directory for the image file, 
                                        # just input the filename if it is placed in the same folder as this notbook
img = tiff.imread(img_path)         # read the image into a 2d array
plt.imshow(img[0])              # show the image, the number represents the number of frame
plt.show()

In [ ]:
f = tp.locate(img[0], 21, invert=False, minmass=20)
# locate the particle in a defined frame (change the number for different frame)
# The number is the estimated size of particle.
# The size must be an odd integer, and it is better to err on the large side.
# invert=False means we are looking for bright spots instead of dark spots
# see later for tuning the min mass

# view locations of particle in the frame
fig_location = tp.annotate(f, img[0]).get_figure()

In [ ]:
# OPTIONAL
# Can be used to tune the parameters

# mass distribution 
fig, ax = plt.subplots()
ax.hist(f['mass'], bins=20)
# Optionally, label the axes.
ax.set(xlabel='mass', ylabel='count')
plt.show()
# min mass can be tuned by changing the number in the particle localisation line

In [ ]:
# OPTIONAL
# Can be used to tune the parameters

# calculate subpixel accuracy (expecting even distribution)
subpx_bias_img = tp.subpx_bias(f)
# We are looking for even distribution here
# If we use a mask size that is too small, the histogram often shows a dip in the middle.

In [ ]:
f = tp.batch(img[:], 21, minmass=20, invert=False)
# locate particles in mutiple frames
# [:] means all frames.

t = tp.link(f, 5, memory=3) 
# the 5 here is the maximal movement of the particle between frames
# memory is the maximal frames allowed for missing particle

t1 = tp.filter_stubs(t, 25) # Filter the by the minimal frames of a single track - set to 25 frames

# Compare the number of particles in the unfiltered and filtered data.
print('Before:', t['particle'].nunique())
print('After:', t1['particle'].nunique())

In [ ]:
fig_massTosize = tp.mass_size(t1.groupby('particle').mean()).get_figure() 
# convenience function -- just plots size vs. mass

In [ ]:
# Plot trajaectories figure
fig_traj = tp.plot_traj(t1).get_figure()

In [ ]:
# Above are the explaination for the steps to locate particles
# I will explain my automation below

In [ ]:
# define a function for easier access
def particle_count(img_path, p_size=21, min_mass=20, max_displace=5, max_miss=3, min_frame=25):
    """
    This function summarise the process of locating particles. 
    *It also saves three figures to the same directory of image stack
    
    para - img_path: path for the .tif image stack
    para - p-size: estimated particle size
    para - min_mass: minimal mass for the particle
    para - max_diplace: maximal movement allowed for particle between frames
    para - max_miss: maximal no. of frames missing allowed
    para - min_frame: minimal no. of frames needed for a particle to be valid
    output - num_of_particle: no. of particles found
    """
    img = tiff.imread(img_path)
    f = tp.locate(img[0], p_size, invert=False)
    
    # plot the localisation of particles in the first frame
    # and save the image with suffix of location
    fig_location = tp.annotate(f, img[0]).get_figure()
    fig_location.savefig(img_path.replace('.tif', '_location.png'))
    
    # process the stack
    f = tp.batch(img[:], p_size, minmass=min_mass, invert=False);
    t = tp.link(f, max_displace, memory=max_miss) 
    t1 = tp.filter_stubs(t, min_frame)
    
    print('Before:', t['particle'].nunique())
    print('After:', t1['particle'].nunique())
    num_of_particle = t1['particle'].nunique()
    
    if num_of_particle != 0:
        # plot the mass vs. size distribution of particles
        # and save the image with suffix of massTosize
        fig_massTosize = tp.mass_size(t1.groupby('particle').mean()).get_figure()
        fig_massTosize.savefig(img_path.replace('.tif', '_massTosize.png'))
        
        # plot the trajaectories
        # and save the image with suffix of traj
        fig_traj = tp.plot_traj(t1).get_figure()
        fig_traj.savefig(img_path.replace('.tif', '_traj.png'))
    
    return num_of_particle

In [ ]:
folder_path = r'C:\Users\zjxia\Documents\Work\zx03_Matthew_Meeting\ThT-Bilayer Data for analyses\638\20191106\PiranhaSurface_1%_2'
# path to the folder containing all images
# containing subfolder won't disrupt the process

output = {} # make a dictionary to contain the results

# walk throught the folder to find all image stacks
for r, d, f in os.walk(folder_path):
    for name in f:
        if name.endswith('.tif'): # filter any file that is not .tif file
            sample_path = os.path.join(r, name)
            # try to locate the particles in the stack
            # if failed, set the result to be zero
            try:
                output[name.replace('_MMStack_Pos0.ome.tif', '')] = particle_count(sample_path)
            except AttributeError or ValueError:
                output[name.replace('_MMStack_Pos0.ome.tif', '')] = 0
output # preview the results
       # run next cell to save the results

In [ ]:
df = pd.DataFrame.from_dict(output, orient ='index')
df = df.reset_index()
df.columns = ['name', 'count']
df.to_csv(folder_path + '\Summary.csv')

In [ ]:
# Please let me know if there is anything not clear.
# For further explaination, please refer to http://soft-matter.github.io/trackpy/dev/tutorial/walkthrough.html